https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/

In [1]:
import numpy as np
import h5py

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, SimpleRNN, GRU, TimeDistributed

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Generate a list of words (including newline)
wordlist = "wordlists/gallic.txt"
words = [x.lower() for x in open(wordlist)]

# Generate the set of unique characters (including newline)
# https://stackoverflow.com/questions/952914/making-a-flat-list-out-of-list-of-lists-in-python
chars = sorted(list(set([char for word in words for char in word])))

VOCAB_SIZE = len(chars)
N_WORDS = len(words)
MAX_WORD_LEN = 12  # maximum company name length

print("vocabulary of", len(chars), "characters, including the \\n:")
print(chars)
print("\nFirst two sample words:")
print(words[0:2])

vocabulary of 26 characters, including the \n:
['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']

First two sample words:
['aballacos\n', 'abalna\n']


In [3]:
ix_to_char = {ix:char for ix, char in enumerate(chars)}
char_to_ix = {char:ix for ix, char in enumerate(chars)}

In [4]:
X = np.zeros((N_WORDS, MAX_WORD_LEN, VOCAB_SIZE))
Y = np.zeros((N_WORDS, MAX_WORD_LEN, VOCAB_SIZE))

for word_i in range(N_WORDS):
    word = words[word_i]
    chars = list(word)
    word_len = len(word)
    
    for char_j in range(min(len(word), MAX_WORD_LEN)):
        char = chars[char_j]
        char_ix = char_to_ix[char]
        X[word_i, char_j, char_ix] = 1
        if char_j > 0:
            Y[word_i, char_j - 1, char_ix] = 1  # the 'next char' at time point char_j

In [5]:
LAYER_NUM = 1
HIDDEN_DIM = 50

model = Sequential()
# model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
model.add(SimpleRNN(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
#for i in range(LAYER_NUM - 1):
#    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, None, 50)          3850      
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 26)          1326      
_________________________________________________________________
activation_1 (Activation)    (None, None, 26)          0         
Total params: 5,176
Trainable params: 5,176
Non-trainable params: 0
_________________________________________________________________


In [57]:
# sample the first character
initial_char_distribution = model.predict(np.zeros((1, 1, VOCAB_SIZE))).flatten()
ix = 0
while ix == 0:  # make sure the initial character is not a newline (i.e. index 0)
    ix = int(np.random.choice(VOCAB_SIZE, size = 1, p = initial_char_distribution))

generated_word = ix_to_char[ix]  # start with first character, then later successively append chars

# sample all remaining characters
for i in range(1, MAX_WORD_LEN):
    

'a'

In [12]:
def generate_text(model):
    initial_char_distribution = model.predict(np.zeros((1, 1, VOCAB_SIZE))).flatten()
    ix = np.random.choice(VOCAB_SIZE - 1, size = 1, p = initial_char_distribution[1:]) + 1  # leave out the index 0 (newline)
    
    y_char = ix_to_char[ix]
    X = np.zeros((1, MAX_WORD_LEN, VOCAB_SIZE))
    for i in range(MAX_WORD_LEN):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
        if ix_to_char[ix[-1]] == "\n":
            break
    print("\n", end="")
    return ('').join(y_char)

In [16]:
NUM_EPOCHS = 500
BATCH_SIZE = N_WORDS

epoch = 0
while epoch < NUM_EPOCHS:
    model.fit(X, Y, batch_size=BATCH_SIZE, verbose=0, epochs=1)
    epoch += 1
    if epoch % 50 == 0:
        generate_text(model)

vertos
quatos
uretos
hertos
dulos
froganos
quaregos
ares
xello
gallos


In [28]:
initial_char_distribution

array([0.00306856, 0.12435216, 0.01173217, 0.01435904, 0.01188214,
       0.19284552, 0.00098659, 0.00843391, 0.00803371, 0.09215052,
       0.00100746, 0.00470732, 0.05310497, 0.01440515, 0.03030822,
       0.08553564, 0.02203017, 0.0015642 , 0.14398098, 0.01034166,
       0.02304427, 0.11663546, 0.0073792 , 0.00188859, 0.01373165,
       0.00249077], dtype=float32)

In [31]:
initial_char_distribution = model.predict(np.zeros((1, 1, VOCAB_SIZE))).flatten()
initial_char_distribution



array([  26, 1257,  116,  146,  112, 1918,   12,   83,   75,  905,    5,
         72,  552,  128,  321,  878,  220,   18, 1439,   90,  234, 1135,
         75,   20,  142,   21])

In [9]:
generate_text(model)

NameError: name 'length' is not defined

### TODO

- Experiment with different [temperatures](https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally)
- Go to codementor.io and have an expert review your code, e.g. from [this guy](https://www.codementor.io/mepstein68)